<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment1_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 2.x

# Dataset

In [79]:
!rm -f train.csv test.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv

--2020-09-06 12:49:29--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10400749 (9.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   9.92M  26.8MB/s    in 0.4s    

2020-09-06 12:49:30 (26.8 MB/s) - ‘train.csv’ saved [10400749/10400749]

--2020-09-06 12:49:30--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133794 (1.1M) [text/plain]
Saving

In [156]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [284]:
import pandas as pd
import numpy as np
import csv
import zipfile
import glob
from string import digits

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ
from sklearn.feature_extraction.text import CountVectorizer
english_stops = set(stopwords.words('english'))

# from keras.models import Model
# from keras.layers import Dense, Input, Dropout, LSTM, Activation
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# from keras.initializers import glorot_uniform

from collections import defaultdict
import math

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,title,abstract,category
0,1901.05003,Quantum Teleportation-Inspired Algorithm for S...,We show that low-depth random quantum circui...,quant-ph
1,1903.08905,RAP-Net: Recurrent Attention Pooling Networks ...,The response selection has been an emerging ...,cs
2,1907.11533,Combinatorial protein-protein interactions on ...,Scaffold proteins organize cellular processe...,q-bio
3,1711.07582,CVXR: An R Package for Disciplined Convex Opti...,CVXR is an R package that provides an object...,stat
4,1910.05833,Analytical Solution of (2+1) Dimensional Dirac...,"In this article, we studied the system of (2...",quant-ph


In [317]:
def read_csv(filename):
    abstract = []
    category = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile )

        for row in csvReader:
            abstract.append(row[2])
            category.append(row[3])

    # category = pd.factorize(category)[0]
    X = np.asarray(abstract)
    Y = np.asarray(category)

    return X, Y

X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

for index in range(10):
  print(X_train[index])
  print(Y_train[index])

abstract
category
  We show that low-depth random quantum circuits can be efficiently simulated by a quantum teleportation-inspired algorithm. By using logical qubits to redirect and teleport the quantum information in quantum circuits, the original circuits can be renormalized to new circuits with a smaller number of logical qubits. We demonstrate the algorithm to simulate several random quantum circuits, including 1D-chain 1000-qubit 42-depth, 2D-grid 125*8-qubit 42-depth and 2D-Bristlecone 72-qubit 32-depth circuits. Our results present a memory-efficient method with a clear physical picture to simulate low-depth random quantum circuits. 
quant-ph
  The response selection has been an emerging research topic due to the growing interest in dialogue modeling, where the goal of the task is to select an appropriate response for continuing dialogues. To further push the end-to-end dialogue model toward real-world scenarios, the seventh Dialog System Technology Challenge (DSTC7) proposed a

In [290]:
def preformat(sentence):
  sentence = sentence.lower()
  sentence = sentence.translate(str.maketrans('', '', digits))
  sentence = re.sub(r"http\S+", "", sentence)
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))
  sentence = sentence.translate ({ord(c): " " for c in "!@#$%^&*()[]{}/<>?\\|`~=_+'"""})

  tokens = word_tokenize(sentence)
  tags = nltk.pos_tag(tokens)

  lemmatizer = WordNetLemmatizer()
  words = ""
  for i, token in enumerate(tokens):
    pos_tag = tags[i][1]
    if pos_tag.startswith('N'):
        lemma = lemmatizer.lemmatize(token, pos=NOUN)
        words += lemma + " "
    elif pos_tag.startswith('V'):
        lemma = lemmatizer.lemmatize(token, pos=VERB)
        words += lemma + " "
    elif pos_tag.startswith('J'):
        lemma = lemmatizer.lemmatize(token, pos=ADJ)
        words += lemma + " "
    else:
        lemma = token
        words += " "
  
  return ' '.join(words.split())

In [291]:
def build_vocab(X):
    """Build vocabulary from dataset

    Args:
        texts (list): list of tokenized sentences
    
    Returns:
        vocab (dict): map from word to index
    """
    vocab = {}
    for sentences in X:
        sentences = preformat(sentences)
        for word in sentences.split():
            # Check if word is a punctuation
            
            if word in string.punctuation:
                continue
            if word not in vocab:
                idx = len(vocab)
                vocab[word] = idx
    return vocab

vocab = build_vocab(X_train)
print(vocab)

{'abstract': 0, 'show': 1, 'lowdepth': 2, 'random': 3, 'quantum': 4, 'circuit': 5, 'be': 6, 'simulate': 7, 'teleportationinspired': 8, 'algorithm': 9, 'use': 10, 'logical': 11, 'qubits': 12, 'redirect': 13, 'teleport': 14, 'information': 15, 'original': 16, 'renormalize': 17, 'new': 18, 'small': 19, 'number': 20, 'demonstrate': 21, 'several': 22, 'include': 23, 'dchain': 24, 'qubit': 25, 'depth': 26, 'dgrid': 27, 'dbristlecone': 28, 'result': 29, 'present': 30, 'memoryefficient': 31, 'method': 32, 'clear': 33, 'physical': 34, 'picture': 35, 'response': 36, 'selection': 37, 'have': 38, 'emerge': 39, 'research': 40, 'topic': 41, 'due': 42, 'grow': 43, 'interest': 44, 'dialogue': 45, 'modeling': 46, 'goal': 47, 'task': 48, 'select': 49, 'appropriate': 50, 'continue': 51, 'further': 52, 'push': 53, 'endtoend': 54, 'model': 55, 'realworld': 56, 'scenario': 57, 'seventh': 58, 'dialog': 59, 'system': 60, 'technology': 61, 'challenge': 62, 'dstc': 63, 'propose': 64, 'track': 65, 'base': 66, 'r

In [303]:

def train_naive_bayes(texts, labels, target_classes, alpha=1):
    """Train a multinomial Naive Bayes model
    """
    ndoc = 0
    nc = defaultdict(int)   # map from a class label to number of documents in the class
    logprior = dict()
    loglikelihood = dict()
    count = defaultdict(int)  # count the occurrences of w in documents of class c

    vocab = build_vocab(texts)
    # Training
    for s, c in zip(texts, labels):
        ndoc += 1
        nc[c] += 1
        for w in s.split():
            if w in vocab:
                count[(w,c)] += 1
    
    vocab_size = len(vocab)
    for c in target_classes:
        logprior[c] = math.log(nc[c]/ndoc)
        sum_ = 0
        for w in vocab.keys():
            if (w,c) not in count: count[(w,c)] = 0
            sum_ += count[(w,c)]
        
        for w in vocab.keys():
            loglikelihood[(w,c)] = math.log( (count[(w,c)] + alpha) / (sum_ + alpha * vocab_size) )
    
    return logprior, loglikelihood, vocab

In [319]:
target_classes = set()
for index in Y_train:
    target_classes.add(index)

print(target_classes)

{'cs', 'stat', 'quant-ph', 'eess', 'q-bio', 'hep-th', 'hep-ph', 'astro-ph', 'cond-mat', 'category', 'math'}


In [320]:
logprior, loglikelihood, vocab = train_naive_bayes(X_train, Y_train, target_classes)

In [321]:
def test_naive_bayes(testdoc, logprior, loglikelihood, target_classes, vocab):
    sum_ = {}
    for c in  target_classes:
        sum_[c] = logprior[c]
        for w in testdoc.split():
            if w in vocab:
                sum_[c] += loglikelihood[(w,c)]
    # sort keys in sum_ by value
    sorted_keys = sorted(sum_.keys(), key=lambda x: sum_[x], reverse=True)
    return sorted_keys[0]

In [324]:
print(target_classes)
print(vocab)
print()
for index in range(10):
    print(str(Y_train[index]))

{'cs', 'stat', 'quant-ph', 'eess', 'q-bio', 'hep-th', 'hep-ph', 'astro-ph', 'cond-mat', 'category', 'math'}
{'abstract': 0, 'show': 1, 'lowdepth': 2, 'random': 3, 'quantum': 4, 'circuit': 5, 'be': 6, 'simulate': 7, 'teleportationinspired': 8, 'algorithm': 9, 'use': 10, 'logical': 11, 'qubits': 12, 'redirect': 13, 'teleport': 14, 'information': 15, 'original': 16, 'renormalize': 17, 'new': 18, 'small': 19, 'number': 20, 'demonstrate': 21, 'several': 22, 'include': 23, 'dchain': 24, 'qubit': 25, 'depth': 26, 'dgrid': 27, 'dbristlecone': 28, 'result': 29, 'present': 30, 'memoryefficient': 31, 'method': 32, 'clear': 33, 'physical': 34, 'picture': 35, 'response': 36, 'selection': 37, 'have': 38, 'emerge': 39, 'research': 40, 'topic': 41, 'due': 42, 'grow': 43, 'interest': 44, 'dialogue': 45, 'modeling': 46, 'goal': 47, 'task': 48, 'select': 49, 'appropriate': 50, 'continue': 51, 'further': 52, 'push': 53, 'endtoend': 54, 'model': 55, 'realworld': 56, 'scenario': 57, 'seventh': 58, 'dialog':

In [328]:
for index in range(1, 20, 1):
  print(str(Y_train[index]) + " " + test_naive_bayes(X_train[index], logprior, loglikelihood, target_classes, vocab))

quant-ph quant-ph
cs cs
q-bio q-bio
stat eess
quant-ph hep-th
q-bio q-bio
hep-ph hep-ph
quant-ph quant-ph
quant-ph quant-ph
hep-ph hep-ph
astro-ph astro-ph
cs cs
eess eess
hep-ph hep-ph
hep-ph hep-ph
eess eess
stat stat
eess eess
math math


In [337]:
predicted_labels = [test_naive_bayes(s, logprior, loglikelihood, target_classes, vocab)
                    for s in X_test]
from sklearn import metrics

print('Accuracy score: %f' % metrics.accuracy_score(Y_test, predicted_labels))

Accuracy score: 0.864136


In [348]:
for c in target_classes:
    print('Evaluation measures for class %s' % c)
    print('  Precision: %f' % metrics.precision_score(Y_test, predicted_labels, average='micro'))
    print('  Recall: %f' % metrics.recall_score(Y_test, predicted_labels, average='micro'))
    print('  F1: %f' % metrics.f1_score(Y_test, predicted_labels, average='micro'))

Evaluation measures for class cs
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class stat
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class quant-ph
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class eess
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class q-bio
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class hep-th
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class hep-ph
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class astro-ph
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class cond-mat
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class category
  Precision: 0.864136
  Recall: 0.864136
  F1: 0.864136
Evaluation measures for class math
  Precision: 0.864136
  Recall: 0.86

In [349]:
print('Macro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='macro'))
print('Micro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='micro'))

Macro-averaged f1: 0.786146
Micro-averaged f1: 0.864136


In [351]:
print(metrics.classification_report(Y_test, predicted_labels, digits=3))

              precision    recall  f1-score   support

    astro-ph      0.941     0.950     0.945       100
    category      0.000     0.000     0.000         1
    cond-mat      0.864     0.890     0.877       100
          cs      0.694     0.770     0.730       100
        eess      0.811     0.730     0.768       100
      hep-ph      0.956     0.870     0.911       100
      hep-th      0.865     0.900     0.882       100
        math      0.910     0.810     0.857       100
       q-bio      0.885     0.920     0.902       100
    quant-ph      0.891     0.900     0.896       100
        stat      0.850     0.910     0.879       100

    accuracy                          0.864      1001
   macro avg      0.788     0.786     0.786      1001
weighted avg      0.866     0.864     0.864      1001



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [352]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_skilearn = count_vect.fit_transform(X_train)
X_test_skilearn = count_vect.transform(Y_train)

In [353]:
print(count_vect.vocabulary_)

{'abstract': 2551, 'we': 36569, 'show': 30776, 'that': 33838, 'low': 20347, 'depth': 10245, 'random': 27900, 'quantum': 27628, 'circuits': 7352, 'can': 6453, 'be': 5045, 'efficiently': 11821, 'simulated': 31011, 'by': 6267, 'teleportation': 33637, 'inspired': 17847, 'algorithm': 3242, 'using': 35721, 'logical': 20238, 'qubits': 27689, 'to': 34195, 'redirect': 28316, 'and': 3569, 'teleport': 33636, 'the': 33842, 'information': 17677, 'in': 17363, 'original': 24421, 'renormalized': 28692, 'new': 23230, 'with': 36837, 'smaller': 31245, 'number': 23812, 'of': 24057, 'demonstrate': 10121, 'simulate': 31010, 'several': 30539, 'including': 17429, '1d': 870, 'chain': 7016, '1000': 293, 'qubit': 27688, '42': 1604, '2d': 1235, 'grid': 15452, '125': 466, 'bristlecone': 6059, '72': 2099, '32': 1346, 'our': 24541, 'results': 28990, 'present': 26679, 'memory': 21313, 'efficient': 11820, 'method': 21457, 'clear': 7478, 'physical': 25739, 'picture': 25779, 'response': 28949, 'selection': 30321, 'has':

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1.0)
clf.fit(X_train_skilearn, X_train)
clf